In [1]:
from lazy_nlp_pipeline import NLP, Pattern as P, TokenPattern as TP, WordPattern as WP

In [2]:
nlp = NLP(project_name='example_patterns')

In [3]:
# Sequence of tokens

pattern = P(
    TP('1'),
    TP('a'),
)


test_texts = [
    '1 a',
    'Something 1 a something',
    'Something Something2',
]

for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('1 a')[0:3 doc=140483887639776]
Span('1 a')[10:13 doc=140483887639392]


In [4]:
# subpatterns

p1 = P(
    TP('1'),
    TP('2'),
)

pattern = P(
    p1,
    TP('a'),
    p1,
)

test_texts = [
    '1 2 a 1 2',
    'Something 1 2 a 1 2 something:::a:b',
    'Something Something2',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('1 2 a 1 2')[0:9 doc=140483887629360]
Span('1 2 a 1 2')[10:19 doc=140483887633008]


In [5]:
# allow_inbetween

pattern = P(
    TP('1'),
    TP('0'),
    TP('0'),
    
    allow_inbetween=TP(':'),
)

test_texts = [
    '1:0::0',
    'Something 1:::::0:0',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('1:0::0')[0:6 doc=140483887638816]
Span('1:::::0:0')[10:19 doc=140483887639248]


In [6]:
# allow_inbetween=None

pattern = P(
    TP('e'),
    TP('2'),
    TP(' '),
    TP('e'),
    TP('4'),
    
    allow_inbetween=None,
)

test_texts = [
    'e2 e4',
    'e 2 e 4',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('e2 e4')[0:5 doc=140483887643856]


In [7]:
# date example

ymd_date = P(
    TP(isnumeric=True, min_len=4, max_len=4),
    TP('-'),
    TP(isnumeric=True, min_len=2, max_len=2),
    TP('-'),
    TP(isnumeric=True, min_len=2, max_len=2),
    
    allow_inbetween=None,
)

dmy_date = P(
    TP(isnumeric=True, min_len=2, max_len=2),
    TP('-'),
    TP(isnumeric=True, min_len=2, max_len=2),
    TP('-'),
    TP(isnumeric=True, min_len=4, max_len=4),
    
    allow_inbetween=None,
)

pattern = P(
    TP('from', ignore_case=True)[0:1],
    ymd_date | dmy_date,
    TP('to', ignore_case=True),
    ymd_date | dmy_date,
)

test_texts = [
    '1999-01-10',
    'From 2001-01-10 to 2009-01-10',
    'Something 10-01-2001 to 2009-01-10 Something2',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('From 2001-01-10 to 2009-01-10')[0:29 doc=140483890578960]
Span('2001-01-10 to 2009-01-10')[5:29 doc=140483890578960]
Span('10-01-2001 to 2009-01-10')[10:34 doc=140483887634784]


In [8]:
# lemmatization

pattern = P(
    WP(lemma='неприбутковий'),
    WP()[1:],
)

test_texts = [
    'Вікіпе́дія (англ. Wikipedia, МФА: [ˌwɪkɪˈpiːdɪə]) — загальнодоступна вільна '
    'багатомовна онлайн-енциклопедія, якою опікується неприбуткова організація «Фонд Вікімедіа».',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('неприбуткова організація')[125:149 doc=140483887642896]


In [9]:
# pos, note false positives

pattern = P(
    WP(pos='ADJF'),
    WP(pos='NOUN'),
)

test_texts = [
    'Вікіпе́дія (англ. Wikipedia, МФА: [ˌwɪkɪˈpiːdɪə]) — загальнодоступна вільна '
    'багатомовна онлайн-енциклопедія, якою опікується неприбуткова організація «Фонд Вікімедіа».',
]
for span in nlp.match_patterns([pattern], texts=test_texts):
    print(span)

Span('загальнодоступна вільна')[52:75 doc=140483887629984]
Span('багатомовна онлайн')[76:94 doc=140483887629984]
Span('багатомовна онлайн-енциклопедія')[76:107 doc=140483887629984]
Span('неприбуткова організація')[125:149 doc=140483887629984]
